# osdatahub Examples - Filtering

This notebook contains examples of how to add filtering to your OS DataHub queries.

In addition to filtering Product, and Extent, it is possible to query the API to only
return features with certain attributes. These filtering methods will make it easier to access
only the features that you care about.

## Features API

Each product contains  different properties that provide important additional detail to the dataset.
The free OpenData products usually contain a lot fewer properties than the paid-for Premium datasets.

If you would like to find out the different properties that are assigned to each Feature, you can use the
`DescribeFeatureType` API endpoint. This endpoint is not currently implemented by the osdatahub Python library,
but you can find out more about calling the API directly [here](https://osdatahub.os.uk/docs/wfs/technicalSpecification).

Here, we will use two examples - one for the free feature "Zoomstack Local Buildings", and one for
the premium feature "Topographic Area".

For Features API, there are a variety of filters to pick from that are [listed in the docs](https://osdatahub.readthedocs.io/en/latest/Utilities.html#module-osdatahub.filters).

These are all OGC Standard XML filter parameters.

There are a couple ways of combining these filters. One is passing multiple to the `add_filters` method, or calling it multiple times. These will be joined using an "and" operator, meaning all given queries must be met for a feature to be returned.

The other way to combine filters is to use either the `filter_and`/`filter_or` methods or bitwise 'and' and 'or' operators, as shown below:

```
filter1, filter2, filter3 = ...

# select features matching any
filter = filter_or(filter1, filter2, filter3)
filter = filter1 | filter2 | filter3
filter = filter1 | filter_or(filter2, filter3)

# select features matching all
filter = filter_and(filter1, filter2, filter3)
filter = filter1 & filter2 & filter3
filter = filter_and(filter1, filter2 & filter3)

# or some other combination of operators
filter = (filter1 | filter2) & filter3
filter = filter_or(filter1 & filter2, filter3)
```

The resulting filter can be passed straight to `add_filters`, like any other filter:

```
features.add_filters(filter1 | filter2)
```

### Example #1 - Zoomstack Local Buildings

The first step in filtering your API query is understanding which properties are available. To do this, we use the
`DescribeFeatureType` API endpoint described in the previous section. We ran the following query to find all available
properties: (remember to add your own API key to the query)

`https://api.os.uk/features/v1/wfs?service=wfs&version=2.0.0&request=DescribeFeatureTypetypeNames=Zoomstack_localbuildings&key=your-api-key-here`

The query returns the following available properties:

* OBJECTID
* UUID
* SHAPE
* SHAPE_Length
* SHAPE_Area

Now that we know what we can filter by, we can begin to construct our query.

In [1]:
# Setup

from osdatahub import Extent
from osdatahub.FeaturesAPI import FeaturesAPI
from osdatahub.filters import *
from os import environ

key = environ.get("OS_API_KEY") # you can also replace this line with the API key itself
crs = "EPSG:27700"
product = "zoomstack_local_buildings"

In [2]:
# Define an extent around Hammersmith, London
W, S = (521202 , 177370)
E, N = (523546 , 179409)
extent = Extent.from_bbox((W, S, E, N), crs=crs)

Now that we have a key, a product, and an extent, we can create a FeaturesAPI object.

As a baseline, we will also run the query and see the number of returned features.


In [3]:
features = FeaturesAPI(key, product, extent)
results = features.query(limit=350)
print("Number features returned: ", len(results["features"]))

Number features returned:  350


Next, we can add filters. We would like to get all buildings between 10m^2 and 50m^2, so we will be using both the `is_greater_than` and `is_less_than` filter.

When we add a filter, we can see that the number of returned features reduces.

In [4]:
# create filters
size_lower_bound = is_greater_than("SHAPE_Area", 10)
size_upper_bound = is_less_than("SHAPE_Area", 50)
# add filters to features object
features.add_filters(size_lower_bound, size_upper_bound)

results = features.query(limit=350)
print("Number features returned: ", len(results["features"]))

Number features returned:  241


If we want to add a third filter, we can simply create a new filter object and add to the FeaturesAPI object again.

In [5]:
# create new filter
length_lower_bound = is_greater_than("SHAPE_Length", 30)
# add filter to features object
features.add_filters(length_lower_bound)

results = features.query(limit=350)
print("Number features returned: ", len(results["features"]))

Number features returned:  23


From this example, we have found how to create filters, how to apply them to a query, and how to use multiple filters at once.

### Example #2 - Topographic Area

Now we have tried to create filters for a free product, we can turn our attention to premium data. Premium Ordnance Survey
products have far more attributes that you can filter by, making this functionality even more powerful.

Take the Topographic Area feature. There are 29 different attributes to filter by, including details about the area type
and metadata about the feature's revision history. The available properties are:

* OBJECTID
* TOID
* FeatureCode
* Version
* VersionDate
* Theme
* ThemeCount
* CalculatedAreaValue
* ChangeDate
* ReasonForChange
* ChangeHistoryCount
* DescriptiveGroup
* DescriptiveGroupCount
* DescriptiveTerm
* DescriptiveTermCount
* Make
* PhysicalLevel
* PhysicalPresence
* style_code
* BHATopoAreaVersion
* BHAProcessDate
* AbsHMin
* AbsH2
* AbsHMax
* RelH2
* RelHMax
* BHAConf
* Shape_Length
* Shape_Area

With all this extra information, we can use even more filters!

In [6]:
crs = "EPSG:27700"
# note: the product name in the python library isn't always exactly the same as the product name on the API
# technical specification. In this case, the API's product is called Topography_TopographicArea
# See all the products in feature_products.py to see if the name has changed.
product = "topographic_area"

In [7]:
# Define an extent around Hammersmith, London
W, S = (521202 , 177370)
E, N = (523546 , 179409)
extent = Extent.from_bbox((W, S, E, N), crs=crs)

In [8]:
features = FeaturesAPI(key, product, extent)
results = features.query(limit=500)
print("Number features returned: ", len(results["features"]))

Number features returned:  500


This time, as well as the size filter demonstrated in example 1, we will add a filter for text. We will be getting all buildings by adding an "is_equal" filter for the make of either "Manmade" and "is_like" for anything containing the word "Park".

In [9]:
# add is_equal and is_like filter
manmade = is_equal("Make", "Manmade")
park = is_like("Make", "*Park*")
size_lower_bound = is_greater_than("SHAPE_Area", 30)
size_upper_bound = is_less_than("SHAPE_Area", 50)

features.add_filters((manmade | park) & size_lower_bound & size_upper_bound)
results = features.query(limit=500)
print("Number features returned: ", len(results["features"]))

Number features returned:  500


Thus, we can use filters for text to get in-depth results from Premium DataHub datasets.
